# Лабораторная 1

## Загружаем данные

In [8]:
import pandas as pd
import numpy as np

In [9]:
df_resto = pd.read_csv('data/casc-resto.csv', sep=';', decimal=',')

In [10]:
df_dem = pd.read_csv('data/CASC_Constant.csv')

In [11]:
df_resto.describe()

,CustomerID,Restaurant,Quantity,SummBasic,SummAfterPointsUsage
count,8.822220e+05,882222.000000,882222.000000,882222.000000,882222.000000
mean,2.809199e+06,468.268804,1.154900,287.725795,260.255589
std,6.178420e+04,279.170967,0.650287,264.555291,251.289517
min,2.728046e+06,40.000000,0.000000,0.500000,-2593.000000
25%,2.754886e+06,333.000000,1.000000,120.000000,110.000000
50%,2.785104e+06,434.000000,1.000000,225.000000,199.000000
75%,2.878900e+06,712.000000,1.000000,379.000000,348.940000
max,2.913132e+06,980.000000,107.000000,29450.000000,29450.000000


In [12]:
df_dem.describe()

,CustomerId,Age
count,1.000000e+04,9953.000000
mean,2.812482e+06,32.894203
std,6.227632e+04,11.954687
min,2.728046e+06,16.000000
25%,2.756356e+06,26.000000
50%,2.794998e+06,31.000000
75%,2.880618e+06,37.000000
max,2.913132e+06,247.000000


## Чистим данные

In [16]:
df_resto = df_resto[df_resto['SummAfterPointsUsage'] > 0]

In [17]:
df_dem['Age'] = list(map(lambda x : df_dem['Age'].mean() if x > 100 else x, df_dem['Age'])) 

## Обрабатываем данные

### Формируем целевую переменную

In [18]:
df_date = df_resto.groupby('CustomerID').apply(lambda x : pd.Series(dict(Y = x.RKDate.max() >= '2017-07-01')))

### Фильтруем записи

In [19]:
df_agg = df_resto[df_resto['RKDate'] < '2017-07-01']

### Получаем агрегированные данные

In [20]:
from datetime import datetime

def get_agg(tmp):
    date = datetime.strptime('2017-07-01', '%Y-%m-%d')
    tmp['Date'] = pd.to_datetime(tmp['RKDate'], format='%Y-%m-%d')
    recency = (date - tmp['Date'].max()).days
    tmp['Day'] = tmp['Date'].dt.day
    tmp['Month'] = pd.to_datetime(tmp['Date']).dt.to_period('M')
    frequency = tmp['Day'].count() / tmp['Month'].nunique()
    monetary_value = tmp['SummAfterPointsUsage'].mean()
    # Выделяем наиболее часто посещаемый день недели.
    tmp['DayOfWeek'] = tmp['Date'].dt.dayofweek
    dayofweek = tmp['DayOfWeek'].value_counts().idxmax()
    # Кодируем регионы.
    tmp['Region'] = list(map(lambda x : 0 if x == 'Москва и Московская область' else 1, tmp['RegionName']))
    region = tmp['Region'].max()
    return pd.Series(dict(Recency = recency, Frequency = frequency, Monetary_Value = monetary_value, DayOfWeek = dayofweek, Region = region))

In [21]:
df_agg = df_agg.groupby('CustomerID').apply(get_agg)

In [22]:
df_agg.describe()

,Recency,Frequency,Monetary_Value,DayOfWeek,Region
count,9938.000000,9938.000000,9938.000000,9938.000000,9938.0
mean,259.387201,8.389211,292.692648,3.638760,1.0
std,254.477647,5.797380,100.690789,1.904195,0.0
min,1.000000,1.000000,69.794811,0.000000,1.0
25%,42.000000,5.285714,231.172078,2.000000,1.0
50%,158.000000,7.000000,289.312627,4.000000,1.0
75%,438.000000,9.577068,347.028846,5.000000,1.0
max,912.000000,84.962963,1429.631210,6.000000,1.0


## Объединяем данные

In [23]:
df_all = pd.merge(left=df_date, right=df_agg, left_on='CustomerID', right_on='CustomerID')

In [24]:
df_all = pd.merge(left=df_all, right=df_dem, left_on='CustomerID', right_on='CustomerId')
df_all['Sex'] = list(map(lambda x : 0 if x == 'Male' else (1 if x == 'Female' else 2), df_all['Sex']))
df_all['Age'] = df_all['Age'].fillna(df_all['Age'].mean())

In [25]:
df_all.describe()

,Recency,Frequency,Monetary_Value,DayOfWeek,Region,CustomerId,Age,Sex
count,9938.000000,9938.000000,9938.000000,9938.000000,9938.0,9.938000e+03,9938.000000,9938.000000
mean,259.387201,8.389211,292.692648,3.638760,1.0,2.812421e+06,32.091205,0.739787
std,254.477647,5.797380,100.690789,1.904195,0.0,6.231510e+04,8.050908,0.647107
min,1.000000,1.000000,69.794811,0.000000,1.0,2.728046e+06,16.000000,0.000000
25%,42.000000,5.285714,231.172078,2.000000,1.0,2.756314e+06,26.000000,0.000000
50%,158.000000,7.000000,289.312627,4.000000,1.0,2.794642e+06,31.000000,1.000000
75%,438.000000,9.577068,347.028846,5.000000,1.0,2.880642e+06,36.000000,1.000000
max,912.000000,84.962963,1429.631210,6.000000,1.0,2.913132e+06,96.000000,2.000000


## Обучаем модель

### Выделяем нужные признаки

In [26]:
X = df_all[['Recency', 'Frequency', 'Monetary_Value', 'DayOfWeek', 'Region', 'Age', 'Sex']]
y = df_all['Y']

### Обучаем логистическую модель

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
reg = LogisticRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

C:\Users\Tuccc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Оцениваем полученную модель

In [28]:
from sklearn.metrics import recall_score, precision_score

print('Score: ', reg.score(X_test, y_test))
print('Precision: ', precision_score(y_pred, y_test))
print('Recall: ', recall_score(y_pred, y_test))

Score:  0.7298792756539235
Precision:  0.829004329004329
Recall:  0.6689956331877729
